In [1]:
import tensorflow as tf
from tensorflow import keras
from collections import deque
from environment import *
import numpy as np
import random
import os

from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense 

os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

2024-05-31 17:35:23.645495: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 17:35:23.677010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 17:35:23.677038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 17:35:23.678019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 17:35:23.683214: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 17:35:23.683611: I tensorflow/core/platform/cpu_feature_guard.cc:1

2024-05-31 17:35:24.393954: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define the DQN agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer=Adam(), loss=MeanSquaredError())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = np.array(random.sample(self.memory, batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [4]:
# Create the environment
env = BJEnvironment()
state_size = env.preprocess_observation(env.observation_space)
action_size = env.action_space.n

# Initialize the DQN agent
agent = DQNAgent(state_size, action_size)

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
print(env.reset())

QC
KC
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
4C
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 20.0, 4.0)


In [ ]:
# Training loop
batch_size = 32
num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for t in range(500):
        # Render the environment (optional)
        env.render()

        # Choose an action
        action = agent.act(state)

        # Perform the action
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])

        # Remember the experience
        agent.remember(state, action, reward, next_state, done)

        # Update the state
        state = next_state

        # Check if episode is finished
        if done:
            break

        # Train the agent
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

QH
4H
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
7D
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


NameError: name 'state_size' is not defined